# CrewAI Workflow: Summarization and Translation with Groq LLM

This notebook demonstrates a simple two-agent workflow using CrewAI:
- Documentation Summarizer produces a concise summary
- Technical Translator converts the summary to Hindi

It showcases how to:
- Configure an LLM provider (Groq) for CrewAI
- Define Agents with roles, goals, and backstories
- Create Tasks with dependencies (translator waits for summarizer)
- Execute the crew and render the results

Prerequisites:
- A valid `GROQ_API_KEY` available as an environment variable
- Python 3.10+ and the packages: `crewai==0.175.0`, `groq==0.31.1`

Tip: If running in a new environment, install deps first in a separate cell.


In [9]:
# !pip install crewai==0.175.0 groq==0.31.1

## Setup and Dependencies

Install the required packages (skip if already installed):

```python
# !pip install crewai==0.175.0 groq==0.31.1
```

Then import the core classes you'll use from CrewAI.


In [ ]:
# Core CrewAI classes for defining agents, tasks, and the crew
from crewai import Agent, Task, Crew, LLM

# Standard library for reading environment variables
import os


## Configure the LLM (Groq)

This notebook uses Groq's hosted LLM via CrewAI.
- Ensure `GROQ_API_KEY` is set in your environment (e.g., in terminal: `setx GROQ_API_KEY "<your-key>"` on Windows, or `export GROQ_API_KEY=<your-key>` on macOS/Linux, then restart your shell).
- Choose the model string supported by Groq. Here we use `groq/llama-3.3-70b-versatile` for a strong general model.

If the key is missing, CrewAI will raise an authentication error when the agent runs.


In [ ]:
# Read the API key from the environment (not used directly here; CrewAI reads it internally)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize the LLM that powers both agents
# Tip: switch models if you need lower latency or cost
llm = LLM(model="groq/llama-3.3-70b-versatile")


## Define Agents and Tasks

We define two agents with distinct roles and goals:
- Documentation Summarizer: writes concise summaries
- Technical Translator: translates the summary into Hindi

Then we create two tasks:
- `summary_task`: produces a short summary of a React hook description
- `translation_task`: runs after `summary_task` (declared via `dependencies`)

Setting `verbose=True` helps inspect step-by-step reasoning and outputs during execution.


In [ ]:
# Create your CrewAI agents with role, main goal/objective, and backstory/personality
# The summarizer writes a concise technical summary
summarizer = Agent(
    role='Documentation Summarizer',  # Agent's job title/function
    goal='Create concise summaries of technical documentation',  # Agent's main objective
    backstory='Technical writer who excels at simplifying complex concepts',  # Agent's background/expertise
    llm=llm,  # LLM that powers your agent
    verbose=True  # Show agent logs and thought process during execution
)

# The translator converts content to Hindi
translator = Agent(
    role='Technical Translator',
    goal='Translate technical documentation to other languages',
    backstory='Technical translator specializing in software documentation',
    llm=llm,
    verbose=True
)

# Define your agents' tasks
# Task 1: Create a short summary for a React hook description
summary_task = Task(
    description='Summarize this React hook documentation:\n\nuseFetch(url) is a custom hook for making HTTP requests. It returns { data, loading, error } and automatically handles loading states.',
    expected_output="A clear, concise summary of the hook's functionality",
    agent=summarizer  # Task assigned to the summarizer agent
)

# Task 2: Translate the summary into Hindi, must occur after Task 1
translation_task = Task(
    description='Translate the summary to Hindi',
    expected_output="Hindi translation of the hook documentation",
    agent=translator,
    dependencies=[summary_task]  # Enforce execution order
)

# Create crew to manage agents and task workflow
crew = Crew(
    agents=[summarizer, translator],  # Agents included in the crew
    tasks=[summary_task, translation_task],  # Tasks to run in order
    verbose=True
)

# Kick off the workflow; the final result will be the translation string
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4fa09196-db8e-4dfb-b678-c1c9dc98e619                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Documentation Summarizer                                                                                │
│                                                                                                                 │
│  Task: Summarize this React hook documentation:                                                                 │
│                                                                                                                 │
│  useFetch(url) is a custom hook for making HTTP requests. It returns { data, loading, error } and               │
│  automatically handles loading states.                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Documentation Summarizer                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The useFetch(url) is a custom hook for making HTTP requests. It returns { data, loading, error } and           │
│  automatically handles loading states. This hook provides a convenient way to manage the state of HTTP          │
│  requests within React components, allowing for a more streamlined and efficient development process by         │
│  handling the complexities of loading states internally. The return object contains three key properties:       │
│  1. **data**: This property holds the response data from the HTTP request. It is typically used to display or   │
│  process the received information within the component.                                                         │
│  2. **loading**: This is a boolean indicator that signifies whether the HTTP request is currently in progress.  │
│  It's useful for displaying loading indicators or disabling interactive elements until the data is loaded.      │
│  3. **error**: If an error occurs during the HTTP request, this property will contain the error details. It     │
│  allows the component to handle or display errors appropriately, enhancing the user experience by providing     │
│  feedback when requests fail.                                                                                   │
│                                                                                                                 │
│  By using the useFetch hook, developers can easily integrate data fetching into their React applications        │
│  without manually handling the intricacies of request states, thus simplifying the development process and      │
│  making the code more readable and maintainable.                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bb34f1bd-58e2-46b5-854c-5db0c528830a                                                                     │
│  Agent: Documentation Summarizer                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Translator                                                                                    │
│                                                                                                                 │
│  Task: Translate the summary to Hindi                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Translator                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  useFetch(url) एक कस्टम हुक है जो HTTP अनुरोधों को बनाने के लिए उपयोग किया जाता है। यह { data, loading, error } लौटाता है और स्वचालित रूप  │
│  से लोडिंग राज्यों को संभालता है। यह हुक रिएक्ट घटकों के भीतर HTTP अनुरोधों की स्थिति को प्रबंधित करने का एक सुविधाजनक तरीका प्रदान करता है, जिससे विकास       │
│  प्रक्रिया अधिक सुव्यवस्थित और कुशल होती है क्योंकि यह लोडिंग राज्यों की जटिलताओं को आंतरिक रूप से संभालता है। रिटर्न ऑब्जेक्ट में तीन प्रमुख गुण होते हैं:         │
│  1. **डेटा**: यह गुण HTTP अनुरोध से प्राप्त डेटा को धारण करता है। यह आमतौर पर घटक के भीतर प्राप्त जानकारी को प्रदर्शित या प्रसंस्कृत करने के लिए    │
│  उपयोग किया जाता है।                                                                                                  │
│  2. **लोडिंग**: यह एक बूलियन संकेतक है जो यह दर्शाता है कि क्या HTTP अनुरोध वर्तमान में進行 है। यह लोडिंग संकेतक प्रदर्शित करने या डेटा लोड होने तक   │
│  इंटरैक्टिव तत्वों को अक्षम करने के लिए उपयोगी है।                                                                             │
│  3. **त्रुटि**: यदि HTTP अनुरोध के दौरान कोई त्रुटि होती है, तो यह गुण त्रुटि विवरण धारण करेगा। यह घटक को त्रुटियों को संभालने या प्रदर्शित करने की      │
│  अनुमति देता है, जिससे उपयोगकर्ता को अनुरोध विफल होने पर प्रतिक्रिया प्रदान करके अनुभव में सुधार होता है।                                       │
│                                                                                                                 │
│  useFetch हुक का उपयोग करके, डेवलपर अपने रिएक्ट अनुप्रयोगों में डेटा फेचिंग को आसानी से एकीकृत कर सकते हैं بدون मैनुअल रूप से अनुरोध राज्यों की जटिलताओं   │
│  को संभालने, जिससे विकास प्रक्रिया को सरल बनाया जा सकता है और कोड अधिक पठनीय और रखरखाव योग्य हो जाता है।                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bb1bfc3c-b2db-4015-9dad-86ce87654e88                                                                     │
│  Agent: Technical Translator                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4fa09196-db8e-4dfb-b678-c1c9dc98e619                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: useFetch(url) एक कस्टम हुक है जो HTTP अनुरोधों को बनाने के लिए उपयोग किया जाता है। यह { data, loading, error } लौटाता  │
│  है और स्वचालित रूप से लोडिंग राज्यों को संभालता है। यह हुक रिएक्ट घटकों के भीतर HTTP अनुरोधों की स्थिति को प्रबंधित करने का एक सुविधाजनक तरीका प्रदान करता    │
│  है, जिससे विकास प्रक्रिया अधिक सुव्यवस्थित और कुशल होती है क्योंकि यह लोडिंग राज्यों की जटिलताओं को आंतरिक रूप से संभालता है। रिटर्न ऑब्जेक्ट में तीन प्रमुख गुण    │
│  होते हैं:                                                                                                          │
│  1. **डेटा**: यह गुण HTTP अनुरोध से प्राप्त डेटा को धारण करता है। यह आमतौर पर घटक के भीतर प्राप्त जानकारी को प्रदर्शित या प्रसंस्कृत करने के लिए    │
│  उपयोग किया जाता है।                                                                                                  │
│  2. **लोडिंग**: यह एक बूलियन संकेतक है जो यह दर्शाता है कि क्या HTTP अनुरोध वर्तमान में進行 है। यह लोडिंग संकेतक प्रदर्शित करने या डेटा लोड होने तक   │
│  इंटरैक्टिव तत्वों को अक्षम करने के लिए उपयोगी है।                                                                             │
│  3. **त्रुटि**: यदि HTTP अनुरोध के दौरान कोई त्रुटि होती है, तो यह गुण त्रुटि विवरण धारण करेगा। यह घटक को त्रुटियों को संभालने या प्रदर्शित करने की      │
│  अनुमति देता है, जिससे उपयोगकर्ता को अनुरोध विफल होने पर प्रतिक्रिया प्रदान करके अनुभव में सुधार होता है।                                       │
│                                                                                                                 │
│  useFetch हुक का उपयोग करके, डेवलपर अपने रिएक्ट अनुप्रयोगों में डेटा फेचिंग को आसानी से एकीकृत कर सकते हैं بدون मैनुअल रूप से अनुरोध राज्यों की जटिलताओं   │
│  को संभालने, जिससे विकास प्रक्रिया को सरल बनाया जा सकता है और कोड अधिक पठनीय और रखरखाव योग्य हो जाता है।                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Run and Render Results

Execute the crew with `crew.kickoff()` and then render the final output returned by the last task. In this workflow, the last task is the Hindi translation produced by the translator agent.


In [ ]:
# Display the result in formatted markdown for better readability
from IPython.display import Markdown, display

# Convert result (a string) to Markdown; useful if the model returns lists/bullets
# Using Markdown improves readability in notebooks
display(Markdown(str(result)))

useFetch(url) एक कस्टम हुक है जो HTTP अनुरोधों को बनाने के लिए उपयोग किया जाता है। यह { data, loading, error } लौटाता है और स्वचालित रूप से लोडिंग राज्यों को संभालता है। यह हुक रिएक्ट घटकों के भीतर HTTP अनुरोधों की स्थिति को प्रबंधित करने का एक सुविधाजनक तरीका प्रदान करता है, जिससे विकास प्रक्रिया अधिक सुव्यवस्थित और कुशल होती है क्योंकि यह लोडिंग राज्यों की जटिलताओं को आंतरिक रूप से संभालता है। रिटर्न ऑब्जेक्ट में तीन प्रमुख गुण होते हैं:
1. **डेटा**: यह गुण HTTP अनुरोध से प्राप्त डेटा को धारण करता है। यह आमतौर पर घटक के भीतर प्राप्त जानकारी को प्रदर्शित या प्रसंस्कृत करने के लिए उपयोग किया जाता है।
2. **लोडिंग**: यह एक बूलियन संकेतक है जो यह दर्शाता है कि क्या HTTP अनुरोध वर्तमान में進行 है। यह लोडिंग संकेतक प्रदर्शित करने या डेटा लोड होने तक इंटरैक्टिव तत्वों को अक्षम करने के लिए उपयोगी है।
3. **त्रुटि**: यदि HTTP अनुरोध के दौरान कोई त्रुटि होती है, तो यह गुण त्रुटि विवरण धारण करेगा। यह घटक को त्रुटियों को संभालने या प्रदर्शित करने की अनुमति देता है, जिससे उपयोगकर्ता को अनुरोध विफल होने पर प्रतिक्रिया प्रदान करके अनुभव में सुधार होता है।

useFetch हुक का उपयोग करके, डेवलपर अपने रिएक्ट अनुप्रयोगों में डेटा फेचिंग को आसानी से एकीकृत कर सकते हैं بدون मैनुअल रूप से अनुरोध राज्यों की जटिलताओं को संभालने, जिससे विकास प्रक्रिया को सरल बनाया जा सकता है और कोड अधिक पठनीय और रखरखाव योग्य हो जाता है।